In [1]:
import torch
from flcore.pflniid_utils.data_utils import read_client_data
from flcore.clients.clientavg import clientAVG

In [2]:
import copy
import argparse
import os
import time
import warnings
import numpy as np
import logging
from flcore.servers.serveravg import FedAvg
from flcore.servers.serverlocal import Local

In [3]:
from utils.custom_loss_class import *
import pickle

In [ ]:
# From clientbase.py
def load_test_data(self, batch_size=None): 
        # Make sure this runs AFTER load_train_data so the data is already loaded in
        print(f"Client{self.ID}: Setting Test DataLoader")
        if batch_size == None:
            batch_size = self.batch_size

        #test_data = read_client_data(self.dataset, self.ID, self.current_update, is_train=False)
        testing_dataset_obj = CustomEMGDataset(self.cond_samples_npy[self.update_upper_bound:,:], self.cond_labels_npy[self.update_upper_bound:,:])
        X_data = torch.Tensor(testing_dataset_obj['x']).type(torch.float32)
        y_data = torch.Tensor(testing_dataset_obj['y']).type(torch.float32)
        testing_data_for_dataloader = [(x, y) for x, y in zip(X_data, y_data)]

        dl = DataLoader(
            dataset=testing_data_for_dataloader,
            batch_size=batch_size, 
            drop_last=False,  # Yah idk if this should be true or false or if it matters...
            shuffle=False) 
        return dl

In [4]:
model = torch.nn.Sequential(torch.nn.Linear(10, 1, bias=False))
for name, param in model.named_parameters():
    print(f"{name}: {param}")

0.weight: Parameter containing:
tensor([[ 0.0752, -0.1182, -0.1825, -0.1810, -0.0480,  0.0073, -0.2458, -0.1464,
          0.0931, -0.0762]], requires_grad=True)


In [5]:
with torch.no_grad():
    #model[0].weight = torch.nn.Parameter(torch.ones_like(model[0].weight))
    #model[0].weight[0, 0] = 2.
    model[0].weight.fill_(0)
    
    for name, param in model.named_parameters():
        print(f"{name}: {param}")

0.weight: Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], requires_grad=True)


In [8]:
model = torch.nn.Linear(10, 1, bias=False)
for name, param in model.named_parameters():
    print(f"{name}: {param}")

weight: Parameter containing:
tensor([[ 0.2077, -0.2590,  0.1117,  0.0130, -0.3067, -0.1483, -0.0893,  0.2726,
          0.1238,  0.1271]], requires_grad=True)


In [10]:
with torch.no_grad():
    #model[0].weight = torch.nn.Parameter(torch.ones_like(model[0].weight))
    #model[0].weight[0, 0] = 2.
    model.weight.fill_(0)
    
    for name, param in model.named_parameters():
        print(f"{name}: {param}")

weight: Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], requires_grad=True)


In [5]:
path = r'C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Data'
cond0_filename = r'\cond0_dict_list.p'
with open(path+cond0_filename, 'rb') as fp:
    cond0_training_and_labels_lst = pickle.load(fp)

In [12]:
update_ix = np.load(r"..\Data\update_ix.npy")

In [38]:
len(cond0_training_and_labels_lst)

14

In [39]:
print(cond0_training_and_labels_lst[0]['training'].shape)
print(cond0_training_and_labels_lst[0]['labels'].shape)

(20770, 64)
(20770, 2)


In [40]:
normalize_emg=False
pca_channels=64
device_channels=64
dt=1/60
normalize_V=False

my_model = torch.nn.Linear(64, 2)

In [41]:
s_temp = torch.tensor(cond0_training_and_labels_lst[0]['training'][:update_ix[1], :], dtype=torch.float32)
p_reference = torch.transpose(torch.tensor(cond0_training_and_labels_lst[0]['labels'][:update_ix[1], :]), 0, 1)

# First, normalize the entire s matrix
if normalize_emg:
    s_normed = s_temp / torch.linalg.norm(s_temp, ord='fro')
    assert (torch.linalg.norm(s_normed, ord='fro')<1.2) and (torch.linalg.norm(s_normed, ord='fro')>0.8)
else:
    s_normed = s_temp
# Apply PCA if applicable
if pca_channels!=device_channels:  # 64 is the number of channels present on the recording armband
    pca = PCA(n_components=pca_channels)
    s = torch.transpose(torch.tensor(pca.fit_transform(s_normed), dtype=torch.float32), 0, 1)
else:
    s = torch.transpose(s_normed, 0, 1)

F = s[:,:-1]
v_actual =  torch.matmul(my_model.weight, s)
p_actual = torch.cumsum(v_actual, dim=1)*dt  # Numerical integration of v_actual to get p_actual
V = (p_reference - p_actual)*dt
if normalize_V:
    V = V/torch.linalg.norm(V, ord='fro')
    assert (torch.linalg.norm(V, ord='fro')<1.2) and (torch.linalg.norm(V, ord='fro')>0.8)
Y = p_reference[:, :-1]  # To match the input

In [42]:
F.shape

torch.Size([64, 1199])

In [43]:
loss = CPHSLoss(F, my_model.weight, V, F.size()[1], 
                     lambdaF=0, lambdaD=1e-3, lambdaE=1e-4, 
                     Nd=2, Ne=64, return_cost_func_comps=True)
loss_log = []
running_epoch_loss = []

In [44]:
running_num_samples = 0
x = torch.transpose(F,0,1)
y = torch.transpose(Y,0,1)
#if type(x) == type([]):
#    x[0] = x[0].to(self.device)
#else:
#    x = x.to(self.device)
#y = y.to(self.device)
output = my_model(x)
#print(f"clientAVG ----> Training LOSS {i}")  # What is this even tellimg me lol
my_loss = loss(output, y, my_model)
loss_log.append(my_loss.item())
running_num_samples += x.size(0)
#optimizer.zero_grad()
#loss.backward()
#optimizer.step()

ERROR: LambdaE*Error_Norm^2: 178.27844634683584
D: LambdaD*Decoder_Norm^2: 0.0006647210102528334
F: LambdaF*EMG_Norm^2: 0


AttributeError: 'tuple' object has no attribute 'item'

In [45]:
my_loss

(tensor(178.2791, dtype=torch.float64, grad_fn=<AddBackward0>),
 tensor(178.2784, dtype=torch.float64, grad_fn=<MulBackward0>),
 tensor(0.0007, grad_fn=<MulBackward0>),
 0)